In [1]:
import json, collections
import pandas as pd
import csv

In [2]:
import pandas as pd
import json

# Read the JSON file
with open('results.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Create an empty list to store the rows
rows = []

# Iterate over each key-value pair in the JSON data
for key, value in data.items():
    # Extract the required fields from the value
    topic = value.get('topic')
    stance = value.get('stance')
    text = value.get('text')
    author = value.get('author')

    # Create a row as a list with the extracted fields
    row = [topic, stance, text, author]

    # Add the row to the list of rows
    rows.append(row)

# Create the DataFrame using the list of rows
df = pd.DataFrame(rows, columns=['Topic', 'Stance', 'Text', 'Author'])

# Print the DataFrame
print(df)


            Topic    Stance  \
0           avion     other   
1           avion     other   
2           avion     other   
3    décroissance  positive   
4       nucléaire     other   
..            ...       ...   
495     nucléaire  positive   
496         other        -1   
497        viande  negative   
498        viande  negative   
499         other        -1   

                                                  Text           Author  
0    🇸🇬 Un ancien Airbus A380 de Singapore Airlines...      airplusnews  
1    Deux Dassault Falcon 900LX pour la Royal Air F...       AeroBuzzfr  
2    🔴 Israël vient d'annoncer l'interdiction des v...      airplusnews  
3    Mais ce volet là est particulièrement importan...      salomesaque  
4    Ce n’est pas le cas de #Malte, qui a peu de pl...  voixdunucleaire  
..                                                 ...              ...  
495  La baisse des émissions carbone, c'est l'élect...      emma_ducros  
496  ... J'y vois une 1ère impasse.

In [3]:
# Remove "topic" and "author" columns
df = df.drop(['Topic', 'Author'], axis=1)

# Rearrange the columns
df = df[['Text', 'Stance']]

# Print the updated DataFrame
print(df)


                                                  Text    Stance
0    🇸🇬 Un ancien Airbus A380 de Singapore Airlines...     other
1    Deux Dassault Falcon 900LX pour la Royal Air F...     other
2    🔴 Israël vient d'annoncer l'interdiction des v...     other
3    Mais ce volet là est particulièrement importan...  positive
4    Ce n’est pas le cas de #Malte, qui a peu de pl...     other
..                                                 ...       ...
495  La baisse des émissions carbone, c'est l'élect...  positive
496  ... J'y vois une 1ère impasse.\nAvec le plan a...        -1
497  Dans une nouvelle enquête #L214 montre des cas...  negative
498  Entre les transactivistes myopes qui s'abonnen...  negative
499  ➡️ Le gaz contribue à la #CriseClimatique\n➡️ ...        -1

[500 rows x 2 columns]


In [4]:
df = df[df['Stance'] != -1]

In [5]:
df.head()

,Text,Stance
0,🇸🇬 Un ancien Airbus A380 de Singapore Airlines...,other
1,Deux Dassault Falcon 900LX pour la Royal Air F...,other
2,🔴 Israël vient d'annoncer l'interdiction des v...,other
3,Mais ce volet là est particulièrement importan...,positive
4,"Ce n’est pas le cas de #Malte, qui a peu de pl...",other


In [6]:
df.shape

(194, 2)

In [7]:
df_papers=df.rename(columns = {'Text': 'Texte', 'Stance' : 'Avis'}) 
df_papers

,Texte,Avis
0,🇸🇬 Un ancien Airbus A380 de Singapore Airlines...,other
1,Deux Dassault Falcon 900LX pour la Royal Air F...,other
2,🔴 Israël vient d'annoncer l'interdiction des v...,other
3,Mais ce volet là est particulièrement importan...,positive
4,"Ce n’est pas le cas de #Malte, qui a peu de pl...",other
...,...,...
492,On a été dans un des seuls resto un peu gastro...,negative
494,@BShirizadeh @rogerseban @BridonneauV @Reclaim...,other
495,"La baisse des émissions carbone, c'est l'élect...",positive
497,Dans une nouvelle enquête #L214 montre des cas...,negative


In [8]:
df_papers.columns.values

array(['Texte', 'Avis'], dtype=object)

In [9]:
df_papers.groupby(['Avis']).count()

,Texte
Avis,
negative,65
other,45
positive,84


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
        train_test_split(df_papers[["Texte"]], df_papers['Avis'], train_size=0.8, test_size=0.2)

In [11]:
X_train.shape 

(155, 1)

In [12]:
X_test.shape 

(39, 1)

In [13]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\amese\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [14]:
import nltk
from nltk.corpus import stopwords
import numpy as np
# Une seule fois :
if False : # Si déjà fit
    nltk.download('wordnet')
    nltk.download('words')
    nltk.download('punkt')    
    nltk.download('averaged_perceptron_tagger')
    nltk.download('brown')
    nltk.download('stopwords')
    nltk.download('omw-1.4')   
    
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob, Word

import string

# Initialisation du "Wordnet Lemmatizer"
lemmatizer = WordNetLemmatizer()
#Pour tester :  print(lemmatizer.lemmatize("bats"))


from nltk.corpus import brown  # Il y a davantage de mots ici
words = set(brown.words())

#words = set(nltk.corpus.words.words()) : pas beaucoup de mots !
stop_words=set(stopwords.words('french')); # ";" pour ne pas avoir les résultats !

In [15]:
def lemma(texte) :
    #renvoie lemmatizer.lemmatize(texte)
    return [lemmatizer.lemmatize(t) for t in word_tokenize(texte) if \
            t.lower() in words and \
            t.lower() not in stop_words \
            # cas des strs spécifiques non filtrés
            and t not in ["''", '--', '1.2', '1/2', '18th', '2-3', '20th', '4.00', '4.2', '``', 'le', 'u']\
            #and t.lower() not in word_tokenize(stop_words).encode() \ # génère un pb de 'byte' ?!
            and t.lower() not in string.punctuation and not t.isdigit()]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec_lemmatise = CountVectorizer(tokenizer=lemma, stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1.0, min_df=0, max_features=None)

# Transformer les données en  bag of words
count_train = count_vec_lemmatise.fit(df_papers["Texte"])
bag_of_words_of_corpus = count_vec_lemmatise.transform(df_papers["Texte"])

# On enlève qq termes inutiles qui nous ont échappés (qui ont été créés par lemmatize)
# Il s'agit d'un Dict de Python.
for terme_a_jeter in ['n', 'u', 'ft'] :
    count_train.vocabulary_.pop(terme_a_jeter, terme_a_jeter+" n'y est ps !")


# Quelques prints 
print("Il y a ", len(count_train.vocabulary_), " termes dans le vocabulaire\n")

# Print Les 50 premiers termes
print("Les 50 premiers termes:\n{}".format(np.array(count_vec_lemmatise.get_feature_names_out()[:50])))
print("\nTous les 50 termes:\n{}".format(np.array(count_vec_lemmatise.get_feature_names_out()[::50]))) # Tous les 50 termes
#print(type(count_train.vocabulary_))
#print("Vocabulary content:\n {}".format(count_train.vocabulary_))

#les 50 premiers mots et leur nbr d'occurrence
print("\nLe nbr d'occurrence des 50 premiers termes :")
print([(k,v)  for k,v in count_train.vocabulary_.items()][:50])

print("\nLe nbr d'occurrence des 50 premiers termes :")
print([(k,v)  for k,v in count_train.vocabulary_.items()][-50:])

C:\Users\amese\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Il y a  262  termes dans le vocabulaire

Les 50 premiers termes:
['1/3' '2/4' 'abandon' 'action' 'ad' 'aggressor' 'ah' 'aide' 'air'
 'airline' 'alliance' 'alors' 'amp' 'animal' 'anticipation' 'article'
 'assurance' "aujourd'hui" 'avant' 'aviation' 'baron' 'base' 'bien'
 'breaking' 'candidate' 'capital' 'car' 'cargo' 'cause' 'certain'
 'certification' 'chance' 'charge' 'chart' 'chef' 'chose' 'chute'
 'citation' 'cite' 'civil' 'clean' 'code' 'coin' 'collection' 'combat'
 'combustible' 'come' 'comment' 'commercial' 'commission']

Tous les 50 termes:
['1/3' 'compare' 'genre' 'million' 'question' 'tropical']

Le nbr d'occurrence des 50 premiers termes :
[('airline', 9), ('hunt', 110), ('deux', 61), ('falcon', 83), ('royal', 214), ('air', 8), ('force', 93), ('provenance', 196), ('pay', 175), ('important', 118), ('car', 26), ('propose', 194), ('solution', 226), ('tout', 243), ('suite', 235), ('transport', 247), ('place', 180), ('jour', 130), ('plus', 183), ('aide', 7), ('production', 191), ('

C:\Users\amese\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [17]:
# On a déjà fait le nécessaire plus hatut. 
# On met un "if False" pour ne pas exécuter sauf ...
# Eventuellement NE pas utiliser : plus lourd à installer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

if True : 
    # Il y a des problèmes avec la ligne suivante sur certaine machines
    from pattern.en import lemma,lexeme
    import nltk
    from sklearn.feature_extraction.text import CountVectorizer

    class LemmaTokenizer(object):
        def __init__(self):
            self.wnl = WordNetLemmatizer()
        def __call__(self, articles):
            #return [lemma(t) for t in word_tokenize(articles) if t.lower() in words]

            return [lemma(t) for t in word_tokenize(articles) if t.lower() in words and \
                    t.lower() not in stop_words \
                    #and t.lower() not in word_tokenize(stop_words).encode() \ # génère un pb de 'byte' ?!
                    # cas des strs spécifiques non filtrés
                    and t not in ["''", '--', '1.2', '1/2', '18th', '2-3', '20th', '4.00', '4.2', '``']\
                    and t.lower() not in string.punctuation and not t.isdigit()]

        
        
    words = set(nltk.corpus.words.words())

    count_vec_lemmatise = CountVectorizer(tokenizer=LemmaTokenizer(), stop_words="english", analyzer='word', 
                                ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None)

    # Transforms the data into a bag of words
    count_train = count_vec_lemmatise.fit(df_papers["Texte"])
    bag_of_words = count_vec_lemmatise.transform(df_papers["Texte"])

    # Print the first 10 features of the count_vec
    print("Les 50 premiers termes:\n{}".format(count_vec_lemmatise.get_feature_names_out()[:50]))
    print("\nTous les 50 termes:\n{}".format(count_vec_lemmatise.get_feature_names_out()[::50])) # Tous les 50 termes
    #print(type(count_train.vocabulary_))
    #print("Vocabulary content:\n {}".format(count_train.vocabulary_))

    #les 50 premiers mots et leur nbr d'occurrence
    print("\nLe nbr d'occurrence des 50 premiers termes :")
    print([(k,v)  for k,v in count_train.vocabulary_.items()][:50])

Les 50 premiers termes:
['abandon' 'abattoir' 'abord' 'abyssal' 'ad' 'aggressor' 'ah' 'aide' 'air'
 'alimentation' 'allemand' 'aller' 'alliance' 'amour' 'animal'
 'anticipation' 'apache' 'assurer' 'augmenter' 'austral' 'aviation' 'ba'
 'base' 'belle' 'bien' 'bon' 'bord' 'bouche' 'bravo' 'break' 'bure' 'ca'
 'canada' 'candidate' 'capital' 'car' 'cargo' 'cause' 'centrale' 'certain'
 'certification' 'chance' 'changement' 'changer' 'charbon' 'charge'
 'chasse' 'chauffer' 'chef' 'choose']

Tous les 50 termes:
['abandon' 'chute' 'exploitation' 'jean' 'opulence' 'repose' 'utile']

Le nbr d'occurrence des 50 premiers termes :
[('hunt', 134), ('falcon', 105), ('royal', 255), ('air', 8), ('force', 115), ('provenance', 230), ('volet', 307), ('important', 140), ('car', 35), ('propose', 228), ('tout', 289), ('suite', 279), ('place', 212), ('si', 264), ('plu', 215), ('aide', 7), ('production', 225), ('comment', 62), ('cause', 37), ('soja', 269), ('retour', 252), ('assurer', 17), ('semble', 261), ('d

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = stopwords.words('english') + stopwords.words('french')
TfIdf_lemmatise = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words=stop_words,\
                                  smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
# max_df = min_df : par default=1.0

corpus_fitted = TfIdf_lemmatise.fit(df_papers["Texte"])
train_lemmatise_transformed = corpus_fitted.transform(X_train["Texte"])
test_lemmatise_transformed = corpus_fitted.transform(X_test["Texte"])

# Une partie de la matrice TDIDF
temp_df=pd.DataFrame(train_lemmatise_transformed.toarray(), columns=TfIdf_lemmatise.get_feature_names_out())

#print(temp_df.columns.values)
print("1) Les 10 dernières lignes de la  matrice TfIdf (en ligne : les indices, en colonne les termes/mots) :")
print(temp_df.tail(10))
print( "-"*50)

df_idf = pd.DataFrame(TfIdf_lemmatise.idf_, index=TfIdf_lemmatise.get_feature_names_out(),columns=["idf_weights"])
print("2) Quelques indice Idf pour certains mots :")
print(df_idf)
print("-"*50)

print("2bis) Le 100 premiers termes : colonnes de la matrice TfIdf")
liste_termes=TfIdf_lemmatise.get_feature_names_out()
print(liste_termes[:100])
print("-"*50)


print("3) Le 100 derniers termes de la matrice = colonnes de la matrice")
liste_termes = TfIdf_lemmatise.get_feature_names_out()
reversed_liste_termes = liste_termes[::-1]
print(reversed_liste_termes[:100])
print("-" * 50)

print("Il y a 2400 documents, 3204 termes ")
print ("4) La matrice sur X_train : \n", train_lemmatise_transformed.toarray())

print("-"*50)
print("5) Les 50 premiers termes du vocabulaire  et leur indices :")
print([(k,v)  for k,v in TfIdf_lemmatise.vocabulary_.items()][:50])

1) Les 10 dernières lignes de la  matrice TfIdf (en ligne : les indices, en colonne les termes/mots) :
     abandon  abattoir  abord  abyssal   ad  aggressor   ah  aide       air  \
145      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
146      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
147      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
148      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
149      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
150      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
151      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
152      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
153      0.0       0.0    0.0      0.0  0.0   0.000000  0.0   0.0  0.000000   
154      0.0       0.0    0.0      0.0  0.0   6.267858  0.0   0.0  4.321948   

     alimentation  ...  ven

C:\Users\amese\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['might', 'must', 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


In [21]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import make_pipeline

if True :
    # On a fait le travaille plus haut et déjà créer fit_train et fit_test
    pipe = make_pipeline(CountVectorizer(), TfidfTransformer()) # Construire un Construct a Pipeline from the given estimators.
    pipe.fit(X_train['Texte'])

    fit_train = pipe.transform(X_train['Texte'])
    fit_train.shape

In [22]:
fit_test = pipe.transform(X_test['Texte'])
print(fit_test.shape)

(39, 1839)


# Algrithmes de classification des tweets

In [23]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(fit_train, y_train)
dt.score(fit_test, y_test)

0.48717948717948717

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf.fit(fit_train, y_train)

RandomForestClassifier(n_estimators=50)

In [25]:
clf.score(fit_test, y_test)

0.5128205128205128

In [26]:
fit_test.shape

(39, 1839)

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
lr.fit(fit_train, y_train)
lr.score(fit_test, y_test)

0.46153846153846156

In [28]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(fit_train, y_train)
mnb.score(fit_test, y_test)

0.46153846153846156

In [29]:
from sklearn.ensemble import RandomForestClassifier
clf20 = RandomForestClassifier(n_estimators=120, max_depth=20)
clf20.fit(fit_train, y_train)
clf20.score(fit_test, y_test)

0.48717948717948717

In [30]:
from sklearn.ensemble import RandomForestClassifier
clfs=[("clf"+str(N)+','+str(depth), RandomForestClassifier(n_estimators=N, max_depth=depth))  for N in range(1,200,50) for depth in range(10,100,20)]
for nom, _clf_ in clfs :
    _clf_.fit(fit_train, y_train)
    print(nom, ' : score = ', _clf_.score(fit_test, y_test))

clf1,10  : score =  0.358974358974359
clf1,30  : score =  0.46153846153846156
clf1,50  : score =  0.48717948717948717
clf1,70  : score =  0.4358974358974359
clf1,90  : score =  0.3076923076923077
clf51,10  : score =  0.41025641025641024
clf51,30  : score =  0.38461538461538464
clf51,50  : score =  0.41025641025641024
clf51,70  : score =  0.48717948717948717
clf51,90  : score =  0.5128205128205128
clf101,10  : score =  0.38461538461538464
clf101,30  : score =  0.4358974358974359
clf101,50  : score =  0.41025641025641024
clf101,70  : score =  0.5128205128205128
clf101,90  : score =  0.48717948717948717
clf151,10  : score =  0.48717948717948717
clf151,30  : score =  0.48717948717948717
clf151,50  : score =  0.46153846153846156
clf151,70  : score =  0.5384615384615384
clf151,90  : score =  0.46153846153846156


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

gbc40 = GradientBoostingClassifier(n_estimators=200, max_depth=40)
gbc40.fit(fit_train, y_train)
gbc40.score(fit_test, y_test)

0.358974358974359

In [32]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modele_one_vs_linear_SVC= OneVsRestClassifier(LinearSVC())
modele_one_vs_linear_SVC.fit(fit_train, y_train)
modele_one_vs_linear_SVC.score(fit_test, y_test)

0.5641025641025641

In [33]:
from sklearn.svm import SVC
modele_one_vs_SVC = OneVsRestClassifier(SVC())
modele_one_vs_SVC.fit(fit_train, y_train)
modele_one_vs_SVC.score(fit_test, y_test)

0.46153846153846156

In [91]:
# s'il faut télécharger des données
# 
nltk_fait = True # A mettre dès le premier download
if not nltk_fait :
    import nltk
    nltk.download('punkt')
    nltk_fait == True

In [92]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
generated_ngrams = ngrams(word_tokenize(X_train.iloc[0,0]), 3, pad_left=True, pad_right=True)
list(generated_ngrams)[:7]

[(None, None, 'RT'),
 (None, 'RT', '@'),
 ('RT', '@', 'fmomboisse'),
 ('@', 'fmomboisse', ':'),
 ('fmomboisse', ':', 'Pour'),
 (':', 'Pour', 'pallier'),
 ('Pour', 'pallier', 'l')]

In [94]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfTransformer

pipe2 = make_pipeline(CountVectorizer(ngram_range=(1, 2)), TfidfTransformer())
pipe2.fit(X_train['Texte'])
fit_train2 = pipe2.transform(X_train['Texte'])
fit_train2.shape

(155, 5633)

In [95]:
cl = pipe2.steps[0]
cl[1].get_feature_names_out()[100:110]

array(['6mw61er4hp', '6qk1s52nyj', '71', '71 il', '750',
       '750 scientifiques', '777', '777 300er', '7r4lhah7n7', '7x'],
      dtype=object)

In [96]:
fit_test2 = pipe2.transform(X_test['Texte'])
fit_test2.shape

(39, 5633)

In [99]:
stop_words = stopwords.words('english') + stopwords.words('french')
count_vec_lemmatise = CountVectorizer(tokenizer=lemma, stop_words=stop_words, analyzer='word', 
                                ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None)
pipe2_bis = make_pipeline(CountVectorizer(tokenizer=LemmaTokenizer(), stop_words="english", analyzer='word', 
                                ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None), TfidfTransformer())
pipe2_bis.fit(X_train['Texte'])
fit_train2_bis = pipe2_bis.transform(X_train['Texte'])
print(fit_train2_bis.shape)

(155, 576)


In [101]:
fit_test2_bis = pipe2_bis.transform(X_test['Texte'])
fit_test2_bis.shape

(39, 576)

In [102]:
from sklearn.linear_model import LogisticRegression
clf2 = LogisticRegression(solver='lbfgs')
clf2.fit(fit_train2, y_train)

LogisticRegression()

In [103]:
clf2.score(fit_test2, y_test)

0.6410256410256411

In [104]:
from sklearn.linear_model import LogisticRegressionCV
clf2_ = LogisticRegressionCV(cv=5)
clf2_.fit(fit_train2, y_train)

LogisticRegressionCV(cv=5)

In [105]:
clf2_.score(fit_test2, y_test)

0.5641025641025641

In [106]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(fit_train2, y_train)
mnb.score(fit_test2, y_test)

0.6153846153846154

In [109]:
print(X_train.keys())
print(X_train['Texte'])

Index(['Texte'], dtype='object')
382    RT @TierrasTaurinas: La loi européenne donne l...
303    C'est d'ailleurs un gros problème. Le consomma...
68     @MisterNounet @gouzou30 On a pas besoin de cen...
432    Airbus livre 36 appareils en octobre @Airbus #...
354    @HeleneThouy @PartiAnimaliste @TF1 Terrine et ...
                             ...                        
219    RT @greenpeacefr: Le nucléaire, garant de notr...
213    @cdion @GoldbergNic @franceinter Vous avez rai...
34     #nucleaire : pour pouvoir continuer à rabâcher...
229    RT @YvesPDB: Vaccin Covid-19 : "L'AVIATION MON...
437    Habitués à monopoliser la parole sur le #nuclé...
Name: Texte, Length: 145, dtype: object


In [110]:
from sklearn.decomposition import TruncatedSVD
pipe_svd = make_pipeline(CountVectorizer(), TruncatedSVD(n_components=300))
pipe_svd.fit(X_train['Texte'])
fit_train_svd = pipe_svd.transform(X_train['Texte'])
fit_train_svd.shape

(145, 145)

In [111]:
import numpy as np
import pandas as pd
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
# Calculate the average Word2Vec vectors for each comment
X = []
for text in df['Texte']:
    words = text.split()
    vectors = []
    for word in words:
        try:
            vector = model[word]
            vectors.append(vector)
        except KeyError:
            pass
    if len(vectors) > 0:
        X.append(np.mean(vectors, axis=0))
    else:
        X.append(np.zeros(300))

# Transform the target variable 'Avis' into a numeric variable
y = df['Avis'].apply(lambda x: 1 if x == 'positif' else 0)

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier using the Word2Vec vectors
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Evaluate the performance of the classifier
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

FileNotFoundError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin.gz'

In [114]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split



# Preprocess the data
stop_words = stopwords.words('english') + stopwords.words('french')
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(df_papers['Texte'])
y = df_papers['Avis'].apply(lambda x: 1 if x == 'positif' else 0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the base models
base_models = [('logreg', LogisticRegression(max_iter=1000)),
               ('nb', MultinomialNB()),
               ('svm', LinearSVC(max_iter=10000)),
               ('rf', RandomForestClassifier())]

# Define the meta-model
meta_model = LogisticRegression(max_iter=1000)

# Create the stacking classifier
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Fit the stacking classifier on the training data
stacking_clf.fit(X_train, y_train)

# Evaluate the performance of the stacking classifier on the testing data
accuracy = stacking_clf.score(X_test, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0